In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.applications.imagenet_utils import preprocess_input
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *

In [ ]:
!pip install -U efficientnet
import efficientnet.keras as efn

In [ ]:
train_txt = pd.read_csv('/kaggle/input/covidx-cxr2/train.txt', sep=" ", header=None)
train_txt.columns= ["Sl.no","File_Name","Status","Source"]
train_txt.head()

In [ ]:
train_txt.drop(["Sl.no","Source"], axis=1, inplace=True)

In [ ]:
test_txt= pd.read_csv('/kaggle/input/covidx-cxr2/test.txt', sep=" ", header=None)
test_txt.columns= ["Sl.no","File_Name","Status","Source"]
test_txt.head()

In [ ]:
test_txt.drop(["Sl.no","Source"], axis=1, inplace=True)

In [ ]:
test_dest_pth= '/kaggle/input/covidx-cxr2/test/'
test_txt['File_Name'] = test_dest_pth + test_txt['File_Name']

In [ ]:
test_txt

In [ ]:
dest_pth= '/kaggle/input/covidx-cxr2/train/'
train_txt['File_Name'] = dest_pth+train_txt['File_Name']

In [ ]:
train_txt['Status'].value_counts()

In [ ]:
neg = train_txt.Status[train_txt.Status.eq("negative")].sample(13992).index
pos = train_txt.Status[train_txt.Status.eq("positive")].sample(16490).index 

train_txt = train_txt.loc[neg.union(pos)]

In [ ]:
train_txt.shape

In [ ]:
train_df, valid_df=train_test_split(train_txt, test_size=0.2, stratify=train_txt["Status"], random_state=123)

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [ ]:
train_df

In [ ]:
valid_df

In [ ]:
import os
print( os.path.abspath("valid_df"))

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory="/kaggle/working/",
x_col="File_Name",
y_col="Status",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(256,256))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=valid_df,
directory="/kaggle/working/",
x_col="File_Name",
y_col="Status",
subset="validation",
batch_size=10,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(256,256))

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_txt,
directory="/kaggle/working/",
x_col="File_Name",
y_col="Status",
batch_size=32,
seed=42,
shuffle=False,
class_mode="binary",
target_size=(256,256))

In [ ]:
!pip install keras-efficientnet
import efficientnet.tfkeras as efnn
base_model = efnn.EfficientNetB0(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(base_model.input,predictions)

In [ ]:
model_final.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
eff_history = model_final.fit_generator(train_generator, validation_data = valid_generator, steps_per_epoch = 100, epochs = 30)

In [ ]:
model_final.evaluate_generator(generator=valid_generator,steps=10)

In [ ]:
valid_df

In [ ]:
model_final.evaluate_generator(generator=test_generator,steps=10)

In [ ]:
pred=model_final.predict_generator(test_generator)

In [ ]:
preds=pred.round(decimals=0)

In [ ]:
len(pred)

In [ ]:
test_txt['Status'].value_counts()

In [ ]:
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(eff_history.history['loss'], label='train')
pyplot.plot(eff_history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(eff_history.history['accuracy'], label='train')
pyplot.plot(eff_history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.tight_layout()
pyplot.show()

In [ ]:
test_generator.class_indices

In [ ]:
test_txt['Status'][test_txt['Status']=='positive']=1.0
test_txt['Status'][test_txt['Status']=='negative']=0.0

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, preds))
mat = confusion_matrix(test_generator.classes, preds)
print('Classification Report')
target_names = ['Negative', 'Positive']
print(classification_report(test_generator.classes, preds, target_names=target_names))

In [ ]:
ax= plt.subplot()
sns.heatmap(mat, annot=True, fmt='g', ax=ax);  

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Negative', 'Positive']); ax.yaxis.set_ticklabels(['Negative', 'Positive']);

In [ ]:
# Picking out a Positive Test Sample
test_txt['File_Name'][112]

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image= image.load_img('/kaggle/input/covidx-cxr2/test/MIDRC-RICORD-1C-419639-000025-39552-1.png'
                           ,target_size =(256,256))
test_image

In [ ]:
preds[112]

In [ ]:
test_txt['File_Name'][253]

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image= image.load_img('/kaggle/input/covidx-cxr2/test/372b2fea-2013-4600-a06f-765c3f054a82.png'
                           ,target_size =(256,256))
test_image

In [ ]:
preds[253]